In [15]:
!pip install colorama


In [57]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import GPT2Tokenizer, GPT2LMHeadModel, RobertaTokenizer, RobertaForMaskedLM

from colorama import Fore, Style, init

import torch
import nltk
import random


In [39]:
from nltk.corpus import stopwords
init(autoreset=True)
# Download stopwords for masking exclusion
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
class Logging:
    def __init__(self, log_file="logfile.txt"):
        """Initialize the Logging class with a log file name."""
        self.log_file = log_file
        with open(self.log_file, "w") as f:
            pass  # Opening in 'w' mode clears the file content

    def _write_to_file(self, message):
        """Helper method to write log messages to the file."""
        with open(self.log_file, "a") as f:
            f.write(message + "\n")

    def info(self, message):
        """Log an informational message."""
        log_message = f"INFO: {message}"
        print(log_message)  # Optionally print the message to the console
        self._write_to_file(log_message)

    def debug(self, message):
        """Log a debug message."""
        log_message = f"DEBUG: {message}"
        print(log_message)  # Optionally print the message to the console
        self._write_to_file(log_message)

    def error(self, message):
        """Log an error message."""
        log_message = f"ERROR: {message}"
        print(log_message)  # Optionally print the message to the console
        self._write_to_file(log_message)

    def warning(self, message):
        """Log a warning message."""
        log_message = f"WARNING: {message}"
        print(log_message)  # Optionally print the message to the console
        self._write_to_file(log_message)

    def critical(self, message):
        """Log a critical error message."""
        log_message = f"CRITICAL: {message}"
        print(log_message)  # Optionally print the message to the console
        self._write_to_file(log_message)

    def set_log_file(self, log_file):
        """Change the log file."""
        self.log_file = log_file


In [25]:
# Assuming these are already initialized globally
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")



In [44]:
# Load RoBERTa (MLM) for emotional depth enhancement
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [100]:
def generate_gpt2_words_cpu(current_lyrics, k, end_token="."):
    """Generate k words with GPT-2, stopping early if end_token is found."""
    logger.info(f"{Fore.BLUE}Step 1: Generating up to {k} words with GPT-2")

    # Tokenize the input
    inputs = gpt2_tokenizer(current_lyrics, return_tensors='pt', truncation=True, max_length=128)
    device = torch.device("cpu")
    gpt2_model.to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate tokens
    outputs = gpt2_model.generate(
        **inputs,
        max_new_tokens=k,
        do_sample=True,
        top_p=0.98,
        temperature=0.95,
        pad_token_id=gpt2_tokenizer.eos_token_id
    )

    # Decode the output tokens
    new_tokens = outputs[0]  # The generated token IDs
    new_words = gpt2_tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

    # Count the number of tokens generated
    num_tokens_generated = len(new_tokens)

    logger.info(f"Generated {num_tokens_generated} tokens: {new_words}")

    # Optional: Check if end_token is in the generated text
    # if end_token in new_words:
    #     end_idx = new_words.index(end_token) + len(end_token)
    #     new_words = new_words[:end_idx]
    #     logger.info(f"End token '{end_token}' detected, stopping generation")

    return new_words, num_tokens_generated


In [101]:
def select_maskable_words(words, fixed):
    """Calculate maskable range and select words for masking."""
    # prompt_length = len(prompt.split())  # Length of the prompt
    # current_length = len(words)  # Length of current generated lyrics
    print(fixed)

    # Calculate the mask range
    # mask_range_end = min(int(r * current_length + current_length), len(words))
    # logger.info(f"{Fore.YELLOW}Step 2: Determining range for masking")
    # logger.info(f"Mask range end determined at index: {mask_range_end}")

    # Ensure the prompt part is not masked
    # words=['a', 'journalist,', 'but', 'I',"Malay"]

    print(words)
    maskable_words=[]

    for i,w in enumerate(words):
      if w.lower() not in stop_words:
        print(fixed+i,w)
        maskable_words.append((fixed+i,w))


    if len(maskable_words)==0:
      for i,w in enumerate(words):
        if w.lower():
          print(fixed+i,w)
          maskable_words.append((fixed+i,w))

    # maskable_words = [(i, w) for i, w in enumerate(words) if w.lower() #not in stop_words
                      # ]

    return maskable_words


In [102]:
def generate_lyrics(artist_name, k=8, max_length=100, r=0.8, end_token="."):
    """Generate lyrics with an end condition."""
    prompt = f"In the style of {artist_name}, generate me lyrics:"
    logger = Logging(f"my_log_{artist_name}.txt")
    current_lyrics = prompt
    prompt_word_index = len(prompt.split())
    additional = (len(current_lyrics.split()) - prompt_word_index) * 0.8
    split_tokens_till_now=0
    while split_tokens_till_now < max_length:
        # Generate k words with GPT-2
        current_lyrics,split_tokens_till_now = generate_gpt2_words_cpu(current_lyrics, k, end_token)

        # Check if end_token is present
        # if end_token in current_lyrics.split()[-1]:  # Check last word/token
        #     logger.info(f"Stopping generation due to end token: {current_lyrics}")
        #     break

        # Mask and refine with RoBERTa
        words = current_lyrics.split()
        maskable_words = select_maskable_words(words[int(prompt_word_index + additional):],
                                              int(prompt_word_index + additional))

        if len(maskable_words) > 3:
            mask_indices = random.sample([i for i, _ in maskable_words], 2)
        elif len(maskable_words) >= 1:
            mask_indices = random.sample([i for i, _ in maskable_words], 1)
        else:
            mask_indices = None

        if mask_indices:
            masked_lyrics = words.copy()
            for idx in mask_indices:
                masked_lyrics[idx] = roberta_tokenizer.mask_token
            masked_text = ' '.join(masked_lyrics)
            logger.info(f"Masked lyrics: {masked_text}")
            current_lyrics = replace_masked_words(masked_text, mask_indices, prompt_word_index)
            logger.info(f"After replacement: {current_lyrics}")

        print("Current Lyrics:", current_lyrics)
        additional = (len(current_lyrics.split()) - prompt_word_index) * 0.8

    logger.info(f"Final lyrics generated: {current_lyrics}")
    return current_lyrics

In [103]:
def replace_masked_words(masked_text, mask_indices, start_index):
    """Replace masked words using RoBERTa."""
    logger.info(f"{Fore.MAGENTA}Step 3: Predicting masked tokens with RoBERTa")

    # Split the full text into words
    full_lyrics = masked_text.split()
    # Get the partial text starting from start_index
    partial_text = " ".join(full_lyrics[start_index:])
    print("Roberta received:", partial_text)

    # Tokenize the partial text
    inputs = roberta_tokenizer(partial_text, return_tensors='pt')

    # Check if GPU is available
    if torch.cuda.is_available():
        device = torch.device("cuda")
        logger.info(f"{Fore.GREEN}Using GPU for inference")
    else:
        device = torch.device("cpu")
        logger.info(f"{Fore.RED}GPU not available, falling back to CPU")

    # Move model and inputs to device
    roberta_model.to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get model predictions
    with torch.no_grad():
        outputs = roberta_model(**inputs)
    predictions = outputs.logits

    # Convert tokenized input back to tokens for reference
    tokenized_input = roberta_tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    print("Tokenized input:", tokenized_input)

    # Adjust mask_indices to relative positions in partial text
    adjusted_indices = [idx - start_index for idx in mask_indices]

    # Replace masked words in the full lyrics
    for orig_idx, adj_idx in zip(mask_indices, adjusted_indices):
        if full_lyrics[orig_idx] == roberta_tokenizer.mask_token:
            # Get the position in tokenized input corresponding to this word
            # Note: This assumes 1:1 word-to-token mapping which might not always hold
            predicted_id = torch.argmax(predictions[0, adj_idx + 1], axis=-1)  # +1 to account for [CLS]
            predicted_word = roberta_tokenizer.decode(predicted_id, skip_special_tokens=True)
            full_lyrics[orig_idx] = predicted_word

    print("Final lyrics:", full_lyrics)
    return ' '.join(full_lyrics)

In [104]:
logger = Logging("my_log.txt")


In [105]:
lyrics=generate_lyrics("Taylor Swift")

INFO: Step 1: Generating up to 8 words with GPT-2
INFO: Generated 19 tokens: In the style of Taylor Swift, generate me lyrics: "My family's always been great friends
9
['"My', "family's", 'always', 'been', 'great', 'friends']
9 "My
10 family's
11 always
13 great
14 friends
INFO: Masked lyrics: In the style of Taylor Swift, generate me lyrics: <mask> family's always been great <mask>
INFO: Step 3: Predicting masked tokens with RoBERTa
Roberta received: <mask> family's always been great <mask>
INFO: GPU not available, falling back to CPU
Tokenized input: ['<s>', '<mask>', 'Ġfamily', "'s", 'Ġalways', 'Ġbeen', 'Ġgreat', '<mask>', '</s>']
Final lyrics: ['In', 'the', 'style', 'of', 'Taylor', 'Swift,', 'generate', 'me', 'lyrics:', 'My', "family's", 'always', 'been', 'great', ' great']
INFO: After replacement: In the style of Taylor Swift, generate me lyrics: My family's always been great  great
Current Lyrics: In the style of Taylor Swift, generate me lyrics: My family's always been great  gr

In [106]:
lyrics

"In the style of Taylor Swift, generate me lyrics: My family's always been great great, I've , so 've but no longer ve so what if I so Then what if you don't then you'll you it all I just don't have know, so what if you don't me know What if you are a don then what if you if no, so what if you don't if what I have , so what if you no me or no me don Then what if you aren't"

In [108]:
tokens = gpt2_tokenizer.encode(lyrics, add_special_tokens=False)
len(tokens)

101

In [110]:
len(lyrics.split())

87